Importação das bibliotecas

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px

Leitura da base de dados

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/enem.csv', encoding='latin-1',sep=';')

Verificar as informações do DataFrame

In [32]:
df.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210053865474,2021,5,F,1,1,1,1,3,1,...,A,A,A,B,A,A,B,A,B,B
1,210052384164,2021,12,M,1,1,1,1,11,1,...,A,A,A,B,A,A,C,A,A,A
2,210052589243,2021,13,F,3,1,1,1,15,1,...,B,A,A,B,A,A,C,B,B,B
3,210052128335,2021,3,M,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,B,B
4,210051353021,2021,2,F,1,3,1,2,0,2,...,B,A,A,B,A,B,E,A,B,B


Verificar quantidade de colunas

In [6]:
len(df.columns)

76

In [33]:
df.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA',
       'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',

## Primeira análise

### Ranking dos estados que obtiveram à média das 1000 maiores notas na redação dos estudantes que concluiram o ensino em escolas públicas 


Inicialmente é necessário obter as redações que estão sem nenhum tipo de problemas, no caso as válidas, o estado da prova, nota da redação e o tipo da escola. Com isso, fazer o ranking das notas das escolas públicas de cada estado e que sejam válidas.

In [7]:
#Fazer uma cópia do dataframe original para modificações
redacao = df[['TP_STATUS_REDACAO','SG_UF_PROVA','NU_NOTA_REDACAO','TP_ESCOLA']].copy()

Filtrar as redações válidas e as notas dos alunos que são de escolas públicas

In [9]:
redacao = redacao.loc[(redacao.TP_STATUS_REDACAO == 1)&(redacao.TP_ESCOLA == 2)]

#deletar as colunas, pois só foram necessárias para filtrar as informações
redacao.drop(['TP_STATUS_REDACAO','TP_ESCOLA'],axis=1,inplace=True)

A linha abaixo permite agrupar os dados por estado e pegar as 1000 maiores notas do enem.

In [10]:
redacao = redacao.groupby('SG_UF_PROVA')['NU_NOTA_REDACAO'].nlargest(1000)

In [11]:
redacao

SG_UF_PROVA         
AC           172563     980.0
             763701     980.0
             838249     980.0
             1017386    980.0
             1082106    980.0
                        ...  
TO           1310325    760.0
             1317663    760.0
             1334585    760.0
             1341276    760.0
             1346949    760.0
Name: NU_NOTA_REDACAO, Length: 27000, dtype: float64

Após fazer esse processo é necessário transformar o conjunto de dados resultante em Dataframe, pois a pós o agrupamento os dados estão em formato de Series.

In [ ]:
df_redacao = pd.DataFrame(redacao)
df_redacao.reset_index()

Com o DataFrame criado, próximo passo é fazer um novo agrupamento que seria a média entre as 1000 maiores notas de cada estado.

In [23]:
df_redacao = df_redacao.groupby(['SG_UF_PROVA'])['NU_NOTA_REDACAO'].mean()

# Pode utilizar a função to_frame também para mudar series para dataframe
df_redacao = df_redacao.to_frame()
df_redacao = df_redacao.reset_index()

In [24]:
# ordenar o dataframe pela média das notas da redacao
df_redacao.sort_values(by=['NU_NOTA_REDACAO'],inplace=True)

In [25]:
df_redacao

,SG_UF_PROVA,NU_NOTA_REDACAO
21,RR,655.94
3,AP,732.76
0,AC,771.24
26,TO,844.82
20,RO,850.92
2,AM,857.16
11,MS,862.34
6,DF,864.40
12,MT,886.56
24,SE,889.22


In [31]:
fig = px.bar(df_redacao, x="NU_NOTA_REDACAO", y="SG_UF_PROVA", orientation='h')
fig.update_layout(title = 'Ranking dos estados que obtiveram à média das 1000 maiores notas do enem no Brasil')
fig.update_xaxes(title = 'Média das 1000 maiores notas da redação')
fig.update_yaxes(title = 'Nome dos Estados' )
fig.show()